Importamos las librerías necesarias

In [4]:
import pandas as pd
import os

Cargamos el archivo

In [5]:
df_production_noflag = pd.read_csv('../../data/raw/Production_Crops_Livestock_E_All_Data_NOFLAG.csv')

Realizamos una epxloración inicial

In [6]:
print("Exploración inicial")
print("Primeras filas:")
print(df_production_noflag.head())
print("\nInformación del dataset:")
print(df_production_noflag.info())
print("\nEstadísticas descriptivas:")
print(df_production_noflag.describe())
print("\nValores nulos por columna:")
print(df_production_noflag.isnull().sum())

Exploración inicial
Primeras filas:
   Area Code Area Code (M49)         Area  Item Code Item Code (CPC)  \
0          2            '004  Afghanistan        221          '01371   
1          2            '004  Afghanistan        221          '01371   
2          2            '004  Afghanistan        221          '01371   
3          2            '004  Afghanistan        711          '01654   
4          2            '004  Afghanistan        711          '01654   

                                                Item  Element Code  \
0                                  Almonds, in shell          5312   
1                                  Almonds, in shell          5412   
2                                  Almonds, in shell          5510   
3  Anise, badian, coriander, cumin, caraway, fenn...          5312   
4  Anise, badian, coriander, cumin, caraway, fenn...          5412   

          Element   Unit  Y1961  ...    Y2014    Y2015    Y2016    Y2017  \
0  Area harvested     ha    0.0  .

Verificamos duplicados

In [7]:
print("\nNúmero de duplicados:", df_production_noflag.duplicated().sum())
df_production_noflag = df_production_noflag.drop_duplicates()


Número de duplicados: 0


Definimos las columnas para mantener como identificadores

In [8]:
id_vars = ['Area Code', 'Area', 'Item Code', 'Item', 'Element Code', 'Element', 'Unit']

Identificamos las columnas de años

In [9]:
year_columns = [col for col in df_production_noflag.columns if col.startswith('Y')]

Cremos un subconjunto con las columnas deseadas

In [10]:
df_production_subset = df_production_noflag[id_vars + year_columns]

Transformamos a formato long

In [11]:
df_clean = pd.melt(
    df_production_subset,
    id_vars=id_vars,
    value_vars=year_columns,
    var_name='Year',
    value_name='Value'
)

Extraemos el año de la columnas 'Year', además de retirar el prefijo 'Y'

In [12]:
df_clean['Year'] = df_clean['Year'].str.extract(r'(\d{4})').astype(int)

Estandarizamos formatos de columnas categóricas

In [13]:
df_clean['Area'] = df_clean['Area'].str.strip().str.title()
df_clean['Item'] = df_clean['Item'].str.strip().str.title()
df_clean['Element'] = df_clean['Element'].str.strip().str.title()

Convertimos tipos de datos

In [14]:
df_clean['Value'] = pd.to_numeric(df_clean['Value'], errors='coerce')

Manejamos valores nulo en 'Value'

In [15]:
df_clean = df_clean.dropna(subset=['Value'])

Guardamos el dataset limpio

In [16]:
df_clean.to_csv('../../data/processed/production_crops_noflag_cleaned.csv', index=False)

Guardamos una versión intermedia

In [17]:
df_production_subset.to_csv('../../data/interim/production_crops_noflag_interim.csv', index=False)

Verificamos el resultado final

In [18]:
print("\nPrimeras filas del dataset limpio:")
print(df_clean.head())
print("\nInformación del dataset limpio:")
print(df_clean.info())


Primeras filas del dataset limpio:
   Area Code         Area  Item Code               Item  Element Code  \
0          2  Afghanistan        221  Almonds, In Shell          5312   
2          2  Afghanistan        221  Almonds, In Shell          5510   
6          2  Afghanistan        515             Apples          5312   
7          2  Afghanistan        515             Apples          5412   
8          2  Afghanistan        515             Apples          5510   

          Element   Unit  Year    Value  
0  Area Harvested     ha  1961      0.0  
2      Production      t  1961      0.0  
6  Area Harvested     ha  1961   2220.0  
7           Yield  kg/ha  1961   6801.8  
8      Production      t  1961  15100.0  

Información del dataset limpio:
<class 'pandas.core.frame.DataFrame'>
Index: 4047502 entries, 0 to 5021792
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Area Code     int64  
 1   Area          object 
 2   Item Code     int64 